In [8]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from geoloc_functions import distance_to_centre, direction_from_centre
from openlocationcode import openlocationcode as olc

In [2]:
# Read recipe inputs
v75_2014_2021_transactions_partial_pluscode = dataiku.Dataset("75_2014_2021_transactions_partial_pluscode")
v75_2014_2021_transactions_partial_pluscode_df = v75_2014_2021_transactions_partial_pluscode.get_dataframe()
iris_75 = dataiku.Dataset("iris_75")
iris_75_df = iris_75.get_dataframe()

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [3]:
v75_2014_2021_transactions_partial_pluscode_df['distance_to_centre']= distance_to_centre(v75_2014_2021_transactions_partial_pluscode_df['latitude'],v75_2014_2021_transactions_partial_pluscode_df['longitude'])

In [4]:
v75_2014_2021_transactions_partial_pluscode_df['direction_from_centre'] = np.vectorize(direction_from_centre)(v75_2014_2021_transactions_partial_pluscode_df['longitude'],v75_2014_2021_transactions_partial_pluscode_df['latitude'])

In [5]:
v75_2014_2021_transactions_partial_pluscode_df

,date_mutation,date_mutation_year,date_mutation_month,date_mutation_day,longitude,latitude,type_local,valeur_fonciere_avg,code_postal,lot1_surface_carrez_sum,surface_reelle_bati_sum,surface_m2_not_null,prix_m2_not_null,nombre_pieces_principales_sum,pluscode_16_first,distance_to_centre,direction_from_centre
0,2014-08-08T00:00:00.000Z,2014,8,8,2.380193,48.893677,Appartement,100948.00,75019,17.78,18.0,17.78,5677.615298,1.0,8FW4V9VJ+F3G28XVR,4.974322,NE
1,2014-08-08T00:00:00.000Z,2014,8,8,2.380198,48.862748,Appartement,230000.00,75011,28.96,27.0,28.96,7941.988950,2.0,8FW4V97J+33WVFVWR,2.851981,E
2,2014-08-08T00:00:00.000Z,2014,8,8,2.380353,48.863002,Appartement,460000.00,75011,60.84,62.0,60.84,7560.815253,3.0,8FW4V97J+64536WXX,2.869965,E
3,2014-08-08T00:00:00.000Z,2014,8,8,2.382581,48.889237,Appartement,120000.00,75019,14.90,15.0,14.90,8053.691275,1.0,8FW4V9QM+M2WGF354,4.679813,NE
4,2014-08-08T00:00:00.000Z,2014,8,8,2.385669,48.841869,Appartement,413716.46,75012,NaN,66.0,66.00,6268.431212,3.0,8FW4R9RP+P7VMW423,3.560694,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289481,2014-08-08T00:00:00.000Z,2014,8,8,2.370992,48.869570,Appartement,427500.00,75011,NaN,45.0,45.00,9500.000000,3.0,8FW4V99C+R9HW5543,2.547685,NE
289482,2014-08-08T00:00:00.000Z,2014,8,8,2.371457,48.852254,Appartement,143000.00,75012,13.68,14.0,13.68,10453.216374,1.0,8FW4V92C+WH44MXXW,2.178391,E
289483,2014-08-08T00:00:00.000Z,2014,8,8,2.373831,48.868441,Appartement,185000.00,75011,20.04,22.0,20.04,9231.536926,2.0,8FW4V99F+9GGP3VXW,2.655914,NE
289484,2014-08-08T00:00:00.000Z,2014,8,8,2.374207,48.867977,Appartement,150000.00,75011,30.83,31.0,30.83,4865.390853,1.0,8FW4V99F+5MW47XXW,2.654584,NE


In [9]:
olc.encode()

TypeError: encode() missing 2 required positional arguments: 'latitude' and 'longitude'

In [0]:
data_to_encode = v75_2014_2021_transactions_partial_pluscode_df[v75_2014_2021_transactions_partial_pluscode_df[]]

In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

transactions_enriched_df = ... # Compute a Pandas dataframe to write into transactions_enriched


# Write recipe outputs
transactions_enriched = dataiku.Dataset("transactions_enriched")
transactions_enriched.write_with_schema(transactions_enriched_df)